# Predict num stops

## data from all sessions

In [ ]:
gc_kwargs = helper_GUAT_vs_TAFT_class.gc_kwargs.copy()

gc_kwargs_temp = {**gc_kwargs,
                    'num_old_ff_per_row': 2,
                    'num_new_ff_per_row': 3}

gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs_temp, monkey_name='monkey_Schro')
gas.retrieve_or_make_combined_info(gc_kwargs_temp, combined_info_exists_ok=True, traj_df_exist_in_GUAT_store_ok=True)
gas.unpack_and_reload_combined_info_back_to_self(gas.combined_info, gas.all_traj_feature_names)

## data from one session

In [ ]:
gc_kwargs = helper_GUAT_vs_TAFT_class.gc_kwargs.copy()

raw_data_folder_path = 'all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330'
gcc = GUAT_collect_info_class.GUATCollectInfoForSession(raw_data_folder_path=raw_data_folder_path, 
                                                        gc_kwargs=gc_kwargs, new_point_index_start=0)
_ = gcc.streamline_process_to_collect_info_from_one_session(GUAT_w_ff_df_exists_ok=True,
                                                            GUAT_info_exists_ok=False)

gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs)
gas.unpack_and_reload_combined_info_back_to_self(gcc.important_info, gcc.all_traj_feature_names)

In [ ]:
# get the frequency of each element in the numpy array gas.output
unique_elements, counts_elements = np.unique(gas.output, return_counts=True)
unique_elements, counts_elements 

## ML

### use current data

In [ ]:
gas.process_current_and_alternative_ff_info()
more_ff_attributes = ['ff_distance', 'ff_angle', 'curv_diff']
ff_last_seen_attributes = ['last_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_last_seen', 'angle_from_monkey_now_to_monkey_when_ff_last_seen']
ff_next_seen_attributes = ['next_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_next_seen', 'angle_from_monkey_now_to_monkey_when_ff_next_seen']


gas.find_input_and_output(add_arc_info=True, add_current_curv_of_traj=True, 
                          ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis', 'time_till_next_visible', 'duration_of_last_vis_period']
                          + ff_last_seen_attributes)

gas.prepare_data_for_machine_learning(traj_ff_attributes_to_add=None)
gas.split_data_to_train_and_test(scaling_data=True)

#bagging = BaggingClassifier(n_estimators=200, max_features=0.9, bootstrap_features=True, bootstrap=True, random_state=42)
#gas.use_machine_learning_model(model=bagging) 

gas.use_machine_learning_model(model=None) #MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=1000, activation='relu', solver='adam', random_state=1))

gas.get_pred_results_df()  
gas.add_additional_info_to_plot()
gas.plot_prediction_results(max_plot_to_make=5, show_reward_boundary=True, use_more_ff_inputs=True, use_more_traj_points=True,
                            predict_num_stops=True,
                            )

In [ ]:
len(gas.num_stops[gas.num_stops==1])/len(gas.num_stops)

In [ ]:
sns.histplot(gas.num_stops)
plt.show()

In [ ]:
current_i = 0
max_plot_to_make = 20

In [ ]:

selected_cases = np.arange(current_i, current_i+max_plot_to_make)
gas.plot_prediction_results(selected_cases=selected_cases, 
                            max_plot_to_make=max_plot_to_make, 
                            show_reward_boundary=True, 
                            use_more_ff_inputs=True, 
                            use_more_traj_points=True,
                            predict_num_stops=True)
current_i += max_plot_to_make

In [ ]:
gas.use_machine_learning_model(model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000))   
gas.get_pred_results_df()  

### loop through num_ff

In [ ]:
# loop
for num_old_ff_per_row in [1, 2, 3]:
    for num_new_ff_per_row in [1, 2, 3]:
        print('num_old_ff_per_row:', num_old_ff_per_row)
        print('num_new_ff_per_row:', num_new_ff_per_row)

        gc_kwargs_temp = {**gc_kwargs,
                            'num_old_ff_per_row': num_old_ff_per_row,
                            'num_new_ff_per_row': num_new_ff_per_row}

        gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs_temp)
        gas.retrieve_or_make_combined_info(gc_kwargs_temp, combined_info_exists_ok=True, traj_df_exist_in_GUAT_store_ok=True)
        gas.unpack_and_reload_combined_info_back_to_self(gas.combined_info, gas.all_traj_feature_names)


        gas.process_current_and_alternative_ff_info()
        more_ff_attributes = ['ff_distance', 'ff_angle', 'curv_diff']
        ff_last_seen_attributes = ['last_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_last_seen', 'angle_from_monkey_now_to_monkey_when_ff_last_seen']
        ff_next_seen_attributes = ['next_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_next_seen', 'angle_from_monkey_now_to_monkey_when_ff_next_seen']


        gas.find_input_and_output(add_arc_info=True, add_current_curv_of_traj=True, 
                                ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis', 'time_till_next_visible', 'duration_of_last_vis_period']
                                + ff_last_seen_attributes)

        gas.prepare_data_for_machine_learning(traj_ff_attributes_to_add=None)
        gas.split_data_to_train_and_test(scaling_data=True)
        gas.use_machine_learning_model(model=None) #MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=1000, activation='relu', solver='adam', random_state=1))

        gas.get_pred_results_df()  


# Basic data

## Retrieve monkey data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
PLAYER = "monkey"
trial_total_num = 2

monkey_name = raw_data_folder_path.split('/')[2]
data_name = raw_data_folder_path.split('/')[3]

data_item = animation_class.AnimationClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()
data_item.make_PlotTrials_args()
data_item.make_or_retrieve_target_clust_last_vis_df()

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe
ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
ff_caught_T_new = data_item.ff_caught_T_new
ff_flash_sorted = data_item.ff_flash_sorted
PlotTrials_args = data_item.PlotTrials_args

target_clust_last_vis_df = data_item.target_clust_last_vis_df
target_clust_last_vis_df['ff_index'] = target_clust_last_vis_df['target_index']


## plotting kwargs

In [ ]:
classic_plot_kwargs = {'player': PLAYER,
                       'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True,
                       'show_eye_positions': True,
                       'show_eye_positions_on_the_right': True,
                       'show_connect_path_eye_positions': True,
                       #=== below is different from animation_plot_kwargs
                       'hitting_arena_edge_ok': False,
                       'trial_too_short_ok': False}

combined_plot_kwargs = {'player': PLAYER,
                        'combined_plot': True,
                        'show_alive_fireflies': False,
                        'show_title': False,
                        'show_start': False}

pretty_null_arc_plot_kwargs = {'player': 'monkey',
        'show_believed_target_positions': True,                       
        'show_stops': True,
        'show_reward_boundary': True,
        'show_scale_bar': True,
        'hitting_arena_edge_ok': True,
        'trial_too_short_ok': True,
        'show_alive_fireflies': False,
        'adjust_xy_limits': True,
        'zoom_in': True,
        'minimal_margin': 60,
        }

animation_plot_kwargs = {
    **classic_plot_kwargs,
    'as_part_of_animation': True,
    'show_eye_positions_on_the_right': False,
    'hitting_arena_edge_ok': True,
    'trial_too_short_ok': True,
    'images_dir': None
}

data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)

## best_arc_df

In [ ]:
ff_dataframe_sub = ff_dataframe[ff_dataframe['ff_angle'].between(-np.pi/4, np.pi/4, inclusive='neither')].copy()
curv = curvature_class.CurvatureOfPath(ff_dataframe_sub, monkey_information, ff_caught_T_new, ff_real_position_sorted,
                                       )
curv.make_curvature_df([-25, 25], curv_of_traj_mode='distance', include_cntr_arc_curv=True,
                       include_opt_arc_curv=True,
                       opt_arc_stop_first_vis_bdry=False,
                       ignore_error=False,)
curvature_df = curv.curvature_df.copy()
curv.find_opt_arc_info_from_curvature_df()

In [ ]:
curv.make_best_arc_df()
curv.get_elements_for_plotting(opt_arc_stop_first_vis_bdry=False, ignore_error=True)
#curv.add_column_monkey_passed_by_to_best_arc_df()

best_arc_df = curv.best_arc_df.copy()
curvature_df = curv.curvature_df.copy()

pseudo_manual_anno, pseudo_manual_anno_long = decision_making_utils.make_pseudo_manual_anno(best_arc_df, monkey_information, ff_caught_T_new)

## combine curvature_df with ff_dataframe

In [ ]:
# combine curvature_df with ff_dataframe so that one can add columns like 'abs_curv_diff' to ff_dataframe
curvature_df_sub = curv.curvature_df[['ff_index', 'point_index', 'curv_diff']]
if 'curv_diff' in ff_dataframe.columns:
    ff_dataframe.drop('curv_diff', axis=1, inplace=True)
ff_dataframe = pd.merge(ff_dataframe, curvature_df_sub, on=['ff_index', 'point_index'], how='left')
# fill na of curv_diff, but randomly make half of them negative
na_index = ff_dataframe['curv_diff'].isna()
ff_dataframe.loc[na_index, 'curv_diff'] = np.random.choice([-1,1], size= na_index.sum()) * 0.6
ff_dataframe['abs_curv_diff']  = ff_dataframe['curv_diff'].abs()

## null_arc_info_for_plotting

In [ ]:
reload(show_null_trajectory)
null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(best_arc_df, monkey_information, column_for_color=None, ignore_error=True)
#null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(best_arc_df, monkey_information, column_for_color='diff_percentile_in_decimal')

# also get null_arc_to_center_info_for_plotting
all_point_index = best_arc_df.point_index.values
all_ff_index = best_arc_df.ff_index.values
remaining_index_of_array, remaining_all_ff_index, remaining_all_point_index = show_null_trajectory.eliminate_invalid_ff_for_null_arc(all_ff_index, all_point_index, ff_real_position_sorted, monkey_information)
null_arc_to_center_info_for_plotting = show_null_trajectory.find_and_package_arc_to_center_info_for_plotting(remaining_all_point_index, remaining_all_ff_index, monkey_information, ff_real_position_sorted,
                                                                                                             ignore_error=True)

# Visualization

## plotly

In [ ]:
for i in [1, 3, 7, 8, 14]:
    print(i)
    current_plotly_polar_plot_kargs, all_ff_dict = plotly_utils_polar.prepare_to_make_one_plotly_polar_plot(i, gas.polar_plots_kwargs, gas.point_index_to_plot, gas.GUAT_joined_ff_info, gas.all_traj_feature_names, 
                                                                                                            more_ff_df=gas.more_ff_df, trajectory_features=gc_kwargs['trajectory_features'])

    fig, customdata_columns = plotly_utils_polar.make_one_plotly_polar_plot(**current_plotly_polar_plot_kargs,
                                                        symbol='group', color='time_since_last_vis', 
                                                        columns_for_annotation=['subgroup', 'curv_diff', 'time_till_next_visible', 'time_since_last_vis'],)
                                                        #columns_for_annotation=['subgroup', 'ff_number', 'group', 'curv_diff'],)
    print(all_ff_dict['more_monkey_info_for_ff_in_past_or_future'])
    fig.show()

## dash

In [ ]:
from dash import Dash, html, dcc, Input, Output, callback
from dash.exceptions import PreventUpdate
import pandas as pd
import plotly.express as px
reload(plotly_utils_polar)
def use_dash(i, gas):
    current_plotly_polar_plot_kargs, all_ff_dict = plotly_utils_polar.prepare_to_make_one_plotly_polar_plot(i, gas.polar_plots_kwargs, gas.point_index_to_plot, gas.GUAT_joined_ff_info, gas.all_traj_feature_names, 
                                                                                            more_ff_df=gas.more_ff_df, trajectory_features=gas.trajectory_features,)
    fig, customdata_columns = plotly_utils_polar.make_one_plotly_polar_plot(**current_plotly_polar_plot_kargs, columns_for_annotation=['subgroup', 'curv_diff', 'time_since_last_vis', 'time_till_next_visible'], 
                                                                            additional_customdata_columns=['ff_number'], color='group')
    initial_hover_data = current_plotly_polar_plot_kargs['ff_df'][customdata_columns].iloc[0].values
    ff_number_index = np.where(np.array(customdata_columns) == 'ff_number')[0][0]

    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    df = all_ff_dict['combined_ff_df'].copy()

    app = Dash(__name__, external_stylesheets=external_stylesheets)

    app.layout = html.Div([

        html.Div([
            dcc.Graph(
                id='main_graph',
                figure = fig,
                hoverData={'points': [{'customdata': initial_hover_data}]} #['Original-Present', 0]}]}
            )
        ], style={'width': '50%', 'display': 'inline-block', 'padding': '0 0'}),
        html.Div([
            dcc.Graph(id='ff_history'),
        ], style={'display': 'inline-block', 'width': '50%'}),
    ])



    @callback(
        Output('ff_history', 'figure'),
        Input('main_graph', 'hoverData'))
    def update_ff_history(hoverData):
        print('hoverData:', hoverData)
        print('hoverData[points][0]:', hoverData['points'][0])
        current_ff_number = hoverData['points'][0]['customdata'][ff_number_index]
        # if current_ff_number is not int
        if not isinstance(current_ff_number, int):
            raise PreventUpdate("No update was made because current_ff_number is not an integer.")
        else:
            dff = df[df['ff_number'] == current_ff_number]
            current_group = dff['group'].iloc[0]
            symbol_maps_for_groups = {'Original': 'circle-dot', 'Alternative': 'star-dot', 'More': 'diamond-dot'}

            title = '<b>{}</b>'.format(current_ff_number)
            current_plotly_polar_plot_kargs_temp = current_plotly_polar_plot_kargs.copy()
            current_plotly_polar_plot_kargs_temp['ff_df'] = dff.copy()
            fig, customdata_columns = plotly_utils_polar.make_one_plotly_polar_plot(**current_plotly_polar_plot_kargs_temp,
                                                                columns_for_annotation=['subgroup', 'ff_distance', 'ff_angle', 'curv_diff', 'time_since_last_vis', 'time_till_next_visible'],
                                                                add_colorbar=True, add_legend=True, add_real_and_predicted_labels=False, size=None,
                                                                size_max=15, symbol='time_label', symbol_map={'Present':symbol_maps_for_groups[current_group]})
            return fig

    app.run(debug=True)
    return 

In [ ]:
use_dash(1, gas)

In [ ]:
use_dash(3, gas)

In [ ]:
use_dash(7, gas)

## Other important distributions

### histplot of durations between stops

In [ ]:
# plot the duration of the three stops
diff_stop_time = GUAT_cluster_df['last_stop_time'] - GUAT_cluster_df['first_stop_time']
sns.histplot(diff_stop_time, bins=20)
plt.show()


# make a line plot of 3 points on x-axis, with y-axis being time since first stop


### histplot of time_till_next_visible

In [ ]:
## histplot of durations between stops

In [ ]:
sns.histplot(gcc.GUAT_cur_ff_info['time_till_next_visible'], bins=20, stat='probability')
plt.title('time till next visible')
plt.show()

### curvature upper lower bounds

In [ ]:
# see clean_curvature_info function

In [ ]:
max(curvature_df.curv_diff.values)

In [ ]:
curv = curvature_class.CurvatureOfPath(ff_dataframe, monkey_information, ff_caught_T_new, ff_real_position_sorted)
curv.make_curvature_df([-1, 1])
curv.find_opt_arc_info_from_curvature_df()
curv.make_best_arc_df()
curv.get_elements_for_plotting()

best_arc_df = curv.best_arc_df.copy()
curvature_df = curv.curvature_df.copy()

In [ ]:
for column in ['curv_of_traj', 'curvature_lower_bound', 'curvature_upper_bound', 'opt_arc_curv']:
    sns.boxplot(curvature_df[column].values, orient='h')
    plt.title(column)
    plt.show()
    
   
    sns.histplot(curvature_df[column].values, bins=50)
    plt.title(column)
    plt.show()
    